In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from survival import DecayMIRModel
from survival import MIRModel
from survival.age_group_extension import AgeSurvivalModel
from survival.three_parameter_log_form import LogFormMIRModel

## INPUT DATA

### [GBD 2019 MIRs with GBD 2019 background mortality]

In [2]:
#PROCESS INPUTS
df = pd.read_csv('/home/j/temp/fed1/2019_mirs_with_background_mortality_usa_only.csv')

#remove NAs and MIR >1
df = df.dropna()
df = df[df['mi_ratio']<=1]

df.head()

,Unnamed: 0,Unnamed: 0.1,location_id,year_id,sex_id,age_group_id,acause,mi_ratio,life_table_parameter_id,background_mortality
0,6605760,6605761,102,1980,1,1,neo_bladder,0.076585,3.0,0.016757
1,6605761,6605762,102,1980,1,1,neo_brain,0.346066,3.0,0.016757
2,6605762,6605763,102,1980,1,1,neo_breast,0.078844,3.0,0.016757
3,6605763,6605764,102,1980,1,1,neo_colorectal,0.267017,3.0,0.016757
4,6605764,6605765,102,1980,1,1,neo_esophageal,0.638391,3.0,0.016757


In [ ]:
df = df[df['location_id']==102]
df.to_csv('/home/j/temp/fed1/2019_mirs_with_background_mortality_usa_only.csv')

In [ ]:
df.head()

## I. Base MIR -> Survival Model:

#### [M/I = (1-P_s^n)(P_c)/(P_c+P_o)]

In [ ]:
#PARAMETERS
num_years = 5
disease_pd = 10

#RUN BASE MODEL
model = MIRModel(df['mi_ratio'],
                 df['background_mortality'],
                 disease_period=disease_pd)
model.compute_excess_mortality()
survival_rate = model.get_survival_rate(num_years=num_years)

# SAVE IN ORIGINAL DATA FRAME
df['excess_mortality_og_mir_model'] = model.excess_mortality
df['abs_survival_rate_og_mir_model'] = survival_rate['abs']
df['rel_survival_rate_og_mir_model'] = survival_rate['rel']

## II. Age Group Extension Model:

In [ ]:
#PARAMETERS
num_years = 5

# RUN AGE EXTENSION MODEL
age_model = AgeSurvivalModel(df)
age_model.compute_P_s()

# Compute the n year survival in model
age_model.compute_n_year_survival(num_years=num_years)

# Compile outputs of model into dataframe
outputs = age_model.combine_outputs()
df.head()

# SAVE IN ORIGINAL DATA FRAME
?

## III. Exponential decay of probability of death model:

#### [log(P_c) = mx + b]

In [ ]:
# PARAMETERS
num_years = 5
df['disease_period'] = 10
df['slope'] = .21

# RUN DECAY MODEL
decay_model = DecayMIRModel(df['mi_ratio'],
                 df['background_mortality'],
                 df['disease_period'],
                 df['slope'])
decay_model.compute_base_excess_mortality()
survival_rate = decay_model.get_survival_rate(num_years=num_years)

# SAVE IN ORIGINAL DATA FRAME
df['base_excess_mortality_decay_model'] = decay_model.base_excess_mortality
df['abs_survival_rate_decay_model'] = survival_rate['abs']
df['rel_survival_rate_decay_model'] = survival_rate['rel']

## IV. Three parameter decay model:

#### [log(P_c) = b*log(1+exp(-(x-a)))+c]

In [3]:
# PARAMETERS
parameters = pd.read_csv('/homes/fed1/Survival_Franny/three_parameter_log_form_values_SEER_6_3_2021.csv')
df['a'] = 5.13551
df['b'] = 0.658518
num_years = 5

# RUN THREE PARAMETER LOG MODEL
three_param_model = LogFormMIRModel(df['mi_ratio'],
                                    df['background_mortality'],
                                    df['a'],
                                    df['b'])
three_param_model.compute_third_parameter()
survival_rate = three_param_model.get_survival_rate(num_years=num_years)

# SAVE IN ORIGINAL DATA FRAME
df['predicted_c'] = three_param_model.c
df['abs_survival_rate_three_param_model'] = survival_rate['abs']
df['rel_survival_rate_three_param_model'] = survival_rate['rel']

-6.221024333216292
-4.506389749514028
-6.1904586995035515
-4.832948688718943
-3.5597375147438015
-5.133159323141854
-5.755619907553182
-5.725109866426692
-5.120235721848303
-5.253136937102454
-3.958874249716473
-5.262168018119837
-3.6926273938240683
-4.0928286636574285
-5.8614173857006335
-3.8143813616636635
-3.671590712086754
-3.0200573019362764
-5.8812661107174335
-3.867182792729409
-5.616167377021384
-4.112722248243017
-5.129230222202068
-4.961620226553123
-5.840554998466132
-3.5917145186215294
-6.449225814805004
-4.2567760649728825
-6.159333021579414
-6.843316486630329
-4.8621645313835415
-6.586020365895138
-7.159619207917558
-6.382793441354124
-4.86916089565625
-5.934356301994512
-5.269251902808492
-5.945868137390562
-4.611354140137109
-6.247552327666881
-4.537836637947521
-6.217018230687472
-4.862656405499592
-3.602539092008783
-5.161739188757792
-5.782754586674628
-5.752295485023069
-5.148856925978862
-5.281359285426106
-3.995233287824931
-5.290365272056761
-3.7328950012795303
-

-4.7429467463337005
-3.7551986551432037
-2.706863871277392
-4.893803158887522
-2.9604279019838486
-3.5599718060177565
-3.954195112672778
-3.266817227396359
-2.9279901115194207
-4.685457355200242
-5.156315898606181
-3.245527632835774
-5.1965332634893
-3.5859108993247446
-4.310812032180138
-4.896121487530536
-5.496482244949448
-2.762426802386244
-6.118411895623694
-3.7605580034562536
-6.311301277618213
-7.012044493384337
-3.882335922197626
-3.6873502797368474
-3.26911747493682
-4.672323930475386
-3.927636291608678
-4.658739955365263
-4.006644678851217
-5.258095916541379
-3.1502003819343827
-5.580099449224463
-4.4383459609514375
-2.764403521651957
-4.06863021100321
-4.709633102605721
-4.987583822929764
-4.755758401655612
-3.796898947535965
-2.7713218479300394
-4.906028597441718
-3.0099880087539077
-3.5853276939417356
-3.8940060585571246
-3.0521112034932685
-2.9408723968720096
-4.5333222130796
-4.993766717205972
-3.280348410565921
-4.984162610211342
-3.6106892014447567
-4.077172063633725
-

-5.652175575042975
-5.652434011855121
-3.725088826644196
-4.134767269675876
-7.389164141724223
-6.887861254703012
-6.103565091981409
-4.546742058047357
-5.710916837944069
-5.5347945327218895
-4.942742062997922
-3.8260952018583794
-4.699717389518128
-6.210538082526682
-6.025625589046513
-4.934726166048904
-5.5553888171209564
-4.080905822153227
-5.442954353369487
-3.7522441586486366
-4.00145393508069
-6.21509089888491
-3.7795486596462524
-3.8571711105248596
-3.1184468809226473
-6.661168105666775
-4.023994757236551
-5.661554131012281
-4.092396812226298
-5.296256737345857
-5.707718621146114
-5.657021606413082
-5.657279957300716
-3.7322179804812095
-4.140912987003276
-7.393741795302731
-6.892475570331271
-6.0824158523348615
-4.145066820711918
-5.689168621905765
-5.5126841658159265
-4.918739767937268
-3.7913768616533616
-4.674480230229392
-6.189513359155399
-5.706769699123697
-4.910688522355713
-4.800068168123062
-4.005975811807048
-5.420625775394683
-3.716058525145627
-3.969636035427584
-5.

-5.007628351100948
-2.9017804275656545
-3.4733851586463516
-3.705217538155239
-2.4894866996680776
-3.057892592338476
-4.3763917634271365
-5.1324724073163
-3.347312508618643
-4.924961489808766
-3.340187062807445
-3.7671864069432104
-4.557080069265567
-4.6458305114958875
-3.5684340046475427
-2.3612222581743936
-3.3582619480068843
-6.443505524222246
-6.047341668346462
-3.8073650697663197
-3.60598260357051
-3.165968574011231
-4.032731614373688
-3.8539242429999376
-4.59792154087495
-3.9349559767890696
-4.621283923460169
-2.3520175042824576
-4.800818774903064
-3.883050844436866
-4.2514631524420565
-2.3981797038288732
-3.1394677533614868
-4.519603935663099
-4.4987302431411536
-4.267557426536541
-3.2655460525859326
-2.356430575817967
-4.814175956545341
-2.5736883769843644
-3.399122112526811
-3.3238660256563435
-1.7235145257961904
-2.8509272665257868
-4.104141070059768
-4.889513885574388
-3.035836858642824
-4.68228494422533
-3.1034891532624442
-3.469096786419641
-4.433266292777489
-4.5033828994

-6.237683679347583
-6.910101252641425
-6.274473510492271
-4.570952175794218
-6.245701915498785
-4.8904130787366284
-3.6247961654328265
-5.186557314242464
-5.837334474362024
-5.776516570984995
-5.180478795708723
-5.345257394163809
-4.042276542962782
-5.322199768598921
-3.7508620693040484
-4.18162901471419
-5.910708909051436
-3.883453035356297
-3.7346847847049958
-3.1021217080353836
-5.952543094430212
-3.918798467960869
-5.653855351505399
-4.177860301637111
-5.216822218034167
-5.02196532136054
-5.908479260771173
-3.6529457574363278
-6.512453511544436
-4.320249692231073
-6.227918056102908
-6.900519847384885
-6.280557613382598
-4.578114697391129
-6.251792607339439
-4.897200178098728
-3.6344611959032136
-5.19309750665002
-5.8435432059252435
-5.782747563517887
-5.187023295997051
-5.351694540480594
-4.050469926763753
-5.32865081885916
-3.7599931103989093
-4.189483720932685
-5.916892594608498
-3.8921129150060096
-3.743879277532187
-3.1154498158059596
-5.9587133320303565
-3.9273460943502414
-5.

...

-3.5527322767563874
-5.913031445940078
-4.260556907387354
-5.594121253769799
-5.949016542225536
-6.152005054397023
-3.280574762549122
-7.060507612342235
-4.444058696559523
-7.069531612695528
-7.632526877629289
-4.2073032098471685
-4.8082903065107985
-4.567582725086368
-5.210766060112064
-4.801861340989717
-4.934916589089653
-4.55806135556827
-6.023872283971059
-4.0488253312503275
-6.031469413750301
-4.966542398709635
-3.15718207294708
-5.089402301747441
-5.9469791552703555
-5.819244565687995
-5.187646225741673
-4.500028345013334
-3.4357286198076054
-5.426409628461215
-3.4298480698441836
-4.686492957681594
-4.625090478426728
-3.8832642245267834
-3.3611087357405722
-5.507881903416774
-6.078254880775376
-3.451800521664836
-5.859659958822476
-4.189760984189661
-5.472362817530438
-5.755203122565717
-6.06278236668345
-3.154851130428824
-7.009557238576463
-4.219790643854822
-6.990942135702198
-7.555803271806099
-4.201596539047614
-4.803261506755234
-4.562337467822317
-5.044193648581086
-4.796

-5.78657498121288
-3.839185527964446
-4.436975588401992
-7.41061604104857
-7.022416494118033
-5.071898816316889
-6.684440942057108
-7.231043385453404
-6.915147124069563
-5.155754379132986
-6.1271176586171805
-5.4995522821184295
-6.0695598164863895
-4.817169702129149
-6.152770792795081
-4.965189125765619
-6.10551492583113
-5.8399731460110615
-5.28887013446024
-3.993206971648533
-4.978475259741799
-6.904739203076815
-6.365160465348936
-5.286614571128894
-6.462942364156561
-4.782859208916079
-5.906230906149403
-3.92130823102093
-4.752928887509142
-6.39444816280843
-4.05913121353592
-4.076848302721601
-3.114981409506754
-6.879564350734881
-3.985764303055248
-5.659914316594268
-4.50402494593277
-5.893561675183037
-5.928847737549177
-6.078682200526373
-5.796511066449739
-3.853221785362848
-4.4487219938980616
-7.420082777872494
-7.031937048144528
-6.159028582827659
-4.972066432102338
-6.111785422643834
-5.846327842560868
-5.295495725932824
-4.0017385505836955
-4.985340389413295
-6.91085941124

-4.041908654918416
-5.206480305261039
-5.849931123919987
-5.726657024107453
-4.744121225190838
-3.1882801312838183
-4.088489588194761
-7.910112720384177
-6.590033584035382
-4.147836552157287
-3.8260376817163433
-3.39253479192962
-4.884179318540873
-4.229533669926013
-4.798054941117016
-4.244228575772695
-5.283316103694915
-3.6666181548969234
-5.685125625456067
-4.776958352164229
-4.899162795189795
-3.459848399512566
-3.9474853718922986
-5.986271865014032
-5.634554730601167
-4.937625659130163
-4.481233573945929
-3.632132538056112
-5.580369923587796
-3.3612941017315734
-4.364574795016551
-4.061879263036872
-3.3331248515594325
-3.508672984793293
-5.394570959173279
-6.217091168354045
-3.4498276168812936
-5.463962381921874
-4.085410508272389
-4.897901685918669
-5.6445582213552825
-5.3832908281049185
-4.414291820554581
-3.0293231155440026
-3.990296990363049
-7.638995850619494
-6.575563011996337
-4.1334235359465765
-3.8099413485622335
-3.3726171000864724
-4.699474285924907
-4.215449561007745


-4.185020667033541
-4.0447102934797385
-3.7103084040643353
-3.2474546932132853
-4.246860157551282
-4.128849339573435
-4.709035289776011
-4.143957440045369
-3.720922791836756
-3.6241718738085638
-3.3921715601434244
-3.985052819670045
-3.9905249656692376
-4.643712149028261
-3.8690225450329763
-3.645662403349661
-3.5455946166291596
-3.3035090548185053
-3.680410337702824
-3.9225273999056753
-4.585837065923215
-3.7980672647120173
-3.519181952093504
-3.4128053730444208
-3.1509558796358306
-3.2649953094198736
-3.8095574092392326
-4.491061701529588
-3.6796439790771274
-3.350972325371231
-3.234250630683846
-2.9381226744487776
-2.624036515159105
-3.662634843884689
-4.371132803031514
-3.5242657062042326
-3.019531566309297
-2.7170101280523795
-4.5043554789674545
-3.0594437676234065
-2.657884959293142
-4.340669233989729
-2.7116913234146978
-4.164029579520729


NameError: name 'decay_model' is not defined

In [5]:
# SAVE IN ORIGINAL DATA FRAME
df['predicted_c'] = three_param_model.c
df['abs_survival_rate_three_param_model'] = survival_rate['abs']
df['rel_survival_rate_three_param_model'] = survival_rate['rel']

In [6]:
df.head()

,Unnamed: 0,Unnamed: 0.1,location_id,year_id,sex_id,age_group_id,acause,mi_ratio,life_table_parameter_id,background_mortality,a,b,predicted_c,abs_survival_rate_three_param_model,rel_survival_rate_three_param_model
0,6605760,6605761,102,1980,1,1,neo_bladder,0.076585,3.0,0.016757,5.13551,0.658518,-6.221024,0.879453,0.941925
1,6605761,6605762,102,1980,1,1,neo_brain,0.346066,3.0,0.016757,5.13551,0.658518,-4.506390,0.676439,0.727902
2,6605762,6605763,102,1980,1,1,neo_breast,0.078844,3.0,0.016757,5.13551,0.658518,-6.190459,0.877811,0.940196
3,6605763,6605764,102,1980,1,1,neo_colorectal,0.267017,3.0,0.016757,5.13551,0.658518,-4.832949,0.737654,0.792507
4,6605764,6605765,102,1980,1,1,neo_esophageal,0.638391,3.0,0.016757,5.13551,0.658518,-3.559738,0.431511,0.468566


## Comparison to Current GBD Methods

#### [Regress on 1-MIR]

In [ ]:
#READ IN AND APPEND OLD SURVIVAL PREDICTIONS TO COMPARE



## Prediction Scatters

In [ ]:
for cancer in df.acause.unique():
        
        data = df[df['acause']==cancer]

        # PLOT
        #t = seer true abs survival value
        t = df['age_group_id']
        plt.plot(t, data['abs_survival_rate_og_mir_model'], 'o', markersize=4, label='base model')
        #plt.plot(t, ?, 'o', markersize=4, label='age extension model')
        plt.plot(t, data['abs_survival_rate_decay_model'], 'o', markersize=4, label='decay model')
        plt.plot(t, data['abs_survival_rate_three_param_model'], 'o', markersize=4, label='three param log model')
        plt.plot(t, data['abs_survival_old_gbd_model'], 'o', markersize=4, label='old GBD method')
        plt.xlabel("t")
        plt.ylabel("abs 5 yr survival")
        plt.legend(loc='lower right')
        plt.title(cancer)
        plt.show()
        

In [ ]:
df.to_csv('/home/j/temp/fed1/comparing_all_survival_methods_6_1_2021.csv')